In [ ]:
import os
import json
import matplotlib.pyplot as plt
import math
%matplotlib notebook

In [ ]:
data_folder = "../../data/convergence_tests"

def load_summary(path):
    with open(data_folder + "/" + path) as file:
        return json.load(file)
    
def load_summaries(directory):
    summary_files = [file for file in os.listdir(directory) if file.endswith(".json")]
    summaries = [ load_summary(os.path.join(directory, filename)) for filename in summary_files ]
    return summaries

In [ ]:
# Draw line depicting convergence rate p
def draw_convergence_line(ax, *, p, x0, y0, label_xoffset):
    # Choose an arbitrary value of x1 in order to determine some point y1
    # which we can give as input to axline
    x1 = 1
    X0 = math.log10(x0)
    Y0 = math.log10(y0)
    X1 = math.log10(x1)
    C = Y0 - p * X0
    Y1 = C + p * X1
    y1 = 10 ** Y1
    ax.axline((x0, y0), (x1, y1), color = "Gray", linestyle="--")
    ax.annotate("$O(h^{})$".format(p), (x0 + label_xoffset, y0))
    
def error_field_from_type(error_type):
    if error_type == 'L2':
        return 'L2_error'
    elif error_type == 'H1_seminorm':
        return 'H1_semi_error'
    else:
        raise "Unknown error type"
        
def label_for_error_type(error_type):
    if error_type == 'L2':
        return '$L^2$ error $|| u - u_h ||_{L^2}$'
    elif error_type == 'H1_seminorm':
        return '$H^1$ seminorm error $| u - u_h |_{H^1}$'
    else:
        raise "Unknown error type"
        
def title_for_error_type(error_type):
    if error_type == 'L2':
        return '$L^2$ error'
    elif error_type == 'H1_seminorm':
        return '$H^1$ seminorm error'
    else:
        raise "Unknown error type"
        
def label_for_fine_res(fine_res):
    if fine_res == 0:
        return "Analytic"
    else:
        return "Fine res = {}".format(fine_res)
    
def prepare_plots(summaries, error_type = 'L2'):
    figs = []

    for summary in summaries:
        fig = plt.figure(figsize=(8, 6), dpi = 128)
        ax = fig.gca()
        
        # Map fine resolutions to samples belonging to this resolution
        fine_to_samples = dict()
        
        for sample in summary['samples']:
            fine_res = int(sample['fine_res'])
            fine_to_samples.setdefault(fine_res,[]).append(sample)
            
        for fine_res, samples in fine_to_samples.items():
            x = [sample['coarse_res'] for sample in samples]
            y = [sample[error_field_from_type(error_type)] for sample in samples]
            ax.semilogy(x, y, label = label_for_fine_res(fine_res), marker = 'o')

        ax.legend(loc = 'upper right')
        ax.set_xlabel("Coarse resolution")
        ax.set_ylabel(label_for_error_type(error_type))
        ax.set_title(title_for_error_type(error_type) + " for " + summary['element_name'])
        ax.grid(visible=True, which='both')
        
        figs.append(fig)

    return figs

In [ ]:
summaries = load_summaries(data_folder + "/error_estimation")
figs = prepare_plots(summaries, error_type='L2')
figs = prepare_plots(summaries, error_type='H1_seminorm')